# Capstone Project - The Battle of Neighborhoods (Week 1)

Install BeautifulSoup 4 

In [1]:
!pip install beautifulsoup4
!pip install geocoder

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd



In [2]:
londonCodes=pd.read_csv('London_postcodes.csv')

In [3]:
londonCodes.columns

Index(['Postcode', 'In Use?', 'Latitude', 'Longitude', 'Easting', 'Northing',
       'Grid Ref', 'County', 'District', 'Ward', 'District Code', 'Ward Code',
       'Country', 'County Code', 'Constituency', 'Introduced', 'Terminated',
       'Parish', 'National Park', 'Population', 'Households', 'Built up area',
       'Built up sub-division', 'Lower layer super output area', 'Rural/urban',
       'Region', 'Altitude', 'London zone', 'LSOA Code', 'Local authority',
       'MSOA Code', 'Middle layer super output area', 'Parish Code',
       'Census output area', 'Constituency Code',
       'Index of Multiple Deprivation', 'Quality', 'User Type', 'Last updated',
       'Nearest station', 'Distance to station', 'Postcode area',
       'Postcode district', 'Police force'],
      dtype='object')

In [4]:
#Remove not assigned from Borough
londonCodes = londonCodes[londonCodes["In Use?"]!="No"]
londonCodes.drop(columns=['Constituency', 'Introduced', 'Terminated','Lower layer super output area'],inplace=True)
londonCodes.drop(columns=['Parish Code','Census output area', 'Constituency Code','Index of Multiple Deprivation'], inplace=True)
londonCodes.drop(columns=['Middle layer super output area', 'Police force','In Use?','County','National Park','Local authority'], inplace=True)
londonCodes.drop(columns=['LSOA Code', 'MSOA Code','Easting','Northing','Grid Ref','Country','Altitude','Quality', 'User Type', 'Last updated'],inplace=True)

In [56]:

londonCodes.rename(columns={'Nearest station':'NearestStation'},inplace = True)
londonCodes.rename(columns={'Distance to station':'DistanceToStation'},inplace = True)

In [71]:
londonCodes.Postcode = londonCodes.Postcode.astype(str)
londonCodes.dtypes

Postcode                  object
Latitude                 float64
Longitude                float64
District                  object
Ward                      object
District Code             object
Ward Code                 object
County Code               object
Parish                    object
Population               float64
Households               float64
Built up area             object
Built up sub-division     object
Rural/urban               object
Region                    object
London zone                int64
NearestStation            object
DistanceToStation        float64
Postcode area             object
Postcode district         object
dtype: object

In [97]:

londonCodes.sort_values(['NearestStation', 'DistanceToStation'], ascending=[True, True], inplace=True)

londonStations = londonCodes.drop_duplicates('NearestStation', keep='first')
londonStations.shape

(638, 20)

2. Use soup to get table with postcodes and also remove rows where Borough is not assigned *

In [99]:
londonStationAll = londonStations

londonStations =  londonStations[['Postcode', 'Latitude', 'Longitude', 'District', 'Ward','NearestStation','DistanceToStation','London zone','Postcode district']]


*** Import required libraries ***

In [100]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library



Now get the coordinates of Toronto, Ontario required for map

In [101]:
address = 'London, UK'

geolocator = Nominatim(user_agent="ln_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

london_lat = latitude
london_long = longitude

The geograpical coordinate of London are 51.5073219, -0.1276474.


Add circle marks for all Boroughs to explore the area 

In [103]:
# create map of Manhattan using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(londonStations['Latitude'], londonStations['Longitude'], londonStations['NearestStation']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [105]:
CLIENT_ID = 'M11SXDXQUL5A5JBJIYK0RXV3IYY1I1ZKFAP43DHNDROFZQMO'#'540243976' # your Foursquare ID
CLIENT_SECRET = 'RICYXPGIS32EPXBDIK0WO2BHRPDSQBOHPPBRY55GVBEGCL3A' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API

In [106]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [107]:
london_venues = getNearbyVenues(names=londonStations['Postcode'],
                                   latitudes=londonStations['Latitude'],
                                   longitudes=londonStations['Longitude']
                                  )

In [152]:
london_venues['Venue Category'].unique()

array(['Trail', 'Light Rail Station', 'Train Station', 'Historic Site',
       'Platform', 'Breakfast Spot', 'Supermarket', 'Convenience Store',
       'Coffee Shop', 'Pub', 'Park', 'Gym / Fitness Center', 'Creperie',
       'Bakery', 'Wine Shop', 'Grocery Store', 'Mini Golf', 'Pool',
       'Gas Station', 'Miscellaneous Shop', 'Hotel', 'Bed & Breakfast',
       'Clothing Store', 'Museum', 'Fish & Chips Shop', 'Pizza Place',
       'Italian Restaurant', 'Fast Food Restaurant', 'Deli / Bodega',
       'American Restaurant', 'Kebab Restaurant', 'Auto Garage',
       'Metro Station', 'Tennis Court', 'Athletics & Sports',
       'Basketball Court', 'Indian Restaurant', 'Café',
       'English Restaurant', 'Tram Station', 'Bus Station',
       'Chinese Restaurant', 'Argentinian Restaurant', 'Cocktail Bar',
       'Japanese Restaurant', 'Lounge', 'Restaurant',
       'Szechuan Restaurant', 'Street Food Gathering', 'Salad Place',
       'Organic Grocery', 'BBQ Joint', 'Beer Bar', 'Asian Resta

In [185]:
filterList = ['Light Rail Station', 'Train Station','Platform','Tram Station', 'Bus Station','Metro Station','Bus Stop']

london_venues = london_venues[~london_venues['Venue Category'].isin(filterList)]


In [186]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Postcode'] = london_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Postcode,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,...,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yakitori Restaurant,Yoga Studio,Yoshoku Restaurant
0,E15 3EB,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,E15 3EB,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,E15 3EB,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,E15 3EB,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,SE2 9RR,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [187]:
london_grouped = london_onehot.groupby('Postcode').mean().reset_index()

In [188]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [189]:
import numpy as np # library to handle data in a vectorized manner

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = london_grouped['Postcode']

for ind in np.arange(london_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BR1 1LX,Coffee Shop,Clothing Store,Sandwich Place,Gym / Fitness Center,Bookstore,Pub,Sushi Restaurant,Fast Food Restaurant,Bakery,Cosmetics Shop
1,BR1 2WE,Home Service,Cricket Ground,Yoshoku Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit
2,BR1 3NN,Pub,Clothing Store,Coffee Shop,Café,Pizza Place,Bar,Burger Joint,Stationery Store,Mexican Restaurant,Donut Shop
3,BR1 3TR,Park,Indian Restaurant,Tennis Court,Café,Falafel Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant
4,BR2 0BX,Construction & Landscaping,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop


In [190]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 10

london_grouped_clustering = london_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 4, 2, 7, 0, 1, 1, 4, 4], dtype=int32)

In [191]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

london_merged = londonStations

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')

london_merged.head() # check the last columns!

,Postcode,Latitude,Longitude,District,Ward,NearestStation,DistanceToStation,London zone,Postcode district,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38933,E15 3EB,51.532409,0.003830,Newham,West Ham,Abbey Road,0.051316,3,E15,4.0,Breakfast Spot,Trail,Historic Site,Yoshoku Restaurant,Farm,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
189769,SE2 9RR,51.490596,0.119759,Greenwich,Abbey Wood,Abbey Wood,0.041189,4,SE2,4.0,Convenience Store,Supermarket,Coffee Shop,Historic Site,Farmers Market,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
302066,W3 6BH,51.508756,-0.263358,Ealing,Acton Central,Acton Central,0.004998,2,W3,8.0,Gym / Fitness Center,Pub,Grocery Store,Creperie,Park,Mini Golf,Coffee Shop,Bakery,Pool,Wine Shop
303280,W3 9EH,51.516545,-0.267674,Ealing,Acton Central,Acton Main Line,0.038011,3,W3,2.0,Convenience Store,Hotel,Miscellaneous Shop,Gas Station,Bed & Breakfast,Grocery Store,Clothing Store,Park,Currency Exchange,English Restaurant
302952,W3 8HN,51.503071,-0.280303,Ealing,South Acton,Acton Town,0.000095,3,W3,4.0,Indian Restaurant,Deli / Bodega,Breakfast Spot,Fish & Chips Shop,Café,Fast Food Restaurant,Tennis Court,Basketball Court,Grocery Store,Auto Garage


In [192]:
london_merged['ClusterLabels'] = london_merged['ClusterLabels'].fillna(0.0).astype(int)
london_merged.head()

,Postcode,Latitude,Longitude,District,Ward,NearestStation,DistanceToStation,London zone,Postcode district,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38933,E15 3EB,51.532409,0.003830,Newham,West Ham,Abbey Road,0.051316,3,E15,4,Breakfast Spot,Trail,Historic Site,Yoshoku Restaurant,Farm,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
189769,SE2 9RR,51.490596,0.119759,Greenwich,Abbey Wood,Abbey Wood,0.041189,4,SE2,4,Convenience Store,Supermarket,Coffee Shop,Historic Site,Farmers Market,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
302066,W3 6BH,51.508756,-0.263358,Ealing,Acton Central,Acton Central,0.004998,2,W3,8,Gym / Fitness Center,Pub,Grocery Store,Creperie,Park,Mini Golf,Coffee Shop,Bakery,Pool,Wine Shop
303280,W3 9EH,51.516545,-0.267674,Ealing,Acton Central,Acton Main Line,0.038011,3,W3,2,Convenience Store,Hotel,Miscellaneous Shop,Gas Station,Bed & Breakfast,Grocery Store,Clothing Store,Park,Currency Exchange,English Restaurant
302952,W3 8HN,51.503071,-0.280303,Ealing,South Acton,Acton Town,0.000095,3,W3,4,Indian Restaurant,Deli / Bodega,Breakfast Spot,Fish & Chips Shop,Café,Fast Food Restaurant,Tennis Court,Basketball Court,Grocery Store,Auto Garage


In [193]:
london_merged.groupby(['ClusterLabels'])
london_merged.head()

,Postcode,Latitude,Longitude,District,Ward,NearestStation,DistanceToStation,London zone,Postcode district,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38933,E15 3EB,51.532409,0.003830,Newham,West Ham,Abbey Road,0.051316,3,E15,4,Breakfast Spot,Trail,Historic Site,Yoshoku Restaurant,Farm,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
189769,SE2 9RR,51.490596,0.119759,Greenwich,Abbey Wood,Abbey Wood,0.041189,4,SE2,4,Convenience Store,Supermarket,Coffee Shop,Historic Site,Farmers Market,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
302066,W3 6BH,51.508756,-0.263358,Ealing,Acton Central,Acton Central,0.004998,2,W3,8,Gym / Fitness Center,Pub,Grocery Store,Creperie,Park,Mini Golf,Coffee Shop,Bakery,Pool,Wine Shop
303280,W3 9EH,51.516545,-0.267674,Ealing,Acton Central,Acton Main Line,0.038011,3,W3,2,Convenience Store,Hotel,Miscellaneous Shop,Gas Station,Bed & Breakfast,Grocery Store,Clothing Store,Park,Currency Exchange,English Restaurant
302952,W3 8HN,51.503071,-0.280303,Ealing,South Acton,Acton Town,0.000095,3,W3,4,Indian Restaurant,Deli / Bodega,Breakfast Spot,Fish & Chips Shop,Café,Fast Food Restaurant,Tennis Court,Basketball Court,Grocery Store,Auto Garage


In [194]:
# create map
latitude = london_lat
longitude = london_long
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['NearestStation'], london_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [195]:
cluster2 = london_merged[london_merged['ClusterLabels'] == 2]
cluster2.head()

,Postcode,Latitude,Longitude,District,Ward,NearestStation,DistanceToStation,London zone,Postcode district,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
303280,W3 9EH,51.516545,-0.267674,Ealing,Acton Central,Acton Main Line,0.038011,3,W3,2,Convenience Store,Hotel,Miscellaneous Shop,Gas Station,Bed & Breakfast,Grocery Store,Clothing Store,Park,Currency Exchange,English Restaurant
126680,N22 7SS,51.598255,-0.119702,Haringey,Bounds Green,Alexandra Palace,0.030864,3,N22,2,Gastropub,Café,Park,Indian Restaurant,Bakery,Bar,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
190461,SE20 8PY,51.412517,-0.065137,Bromley,Penge and Cator,Anerley,0.000000,4,SE20,2,Music Store,Gas Station,Hardware Store,Park,Pub,Flower Shop,Fabric Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant
111358,N11 1AN,51.616221,-0.134285,Enfield,Southgate Green,Arnos Grove,0.001442,4,N11,2,Park,Pool,Chinese Restaurant,Beer Bar,Grocery Store,Flea Market,Flower Shop,Electronics Store,Empanada Restaurant,English Restaurant
253240,TW14 8QA,51.439016,-0.454484,Hounslow,Bedfont,Ashford (Middlesex),0.981598,6,TW14,2,Construction & Landscaping,Park,Gym,Lake,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service


In [196]:
cluster4 = london_merged[london_merged['ClusterLabels'] == 4]
cluster4.head()


,Postcode,Latitude,Longitude,District,Ward,NearestStation,DistanceToStation,London zone,Postcode district,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38933,E15 3EB,51.532409,0.003830,Newham,West Ham,Abbey Road,0.051316,3,E15,4,Breakfast Spot,Trail,Historic Site,Yoshoku Restaurant,Farm,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
189769,SE2 9RR,51.490596,0.119759,Greenwich,Abbey Wood,Abbey Wood,0.041189,4,SE2,4,Convenience Store,Supermarket,Coffee Shop,Historic Site,Farmers Market,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
302952,W3 8HN,51.503071,-0.280303,Ealing,South Acton,Acton Town,0.000095,3,W3,4,Indian Restaurant,Deli / Bodega,Breakfast Spot,Fish & Chips Shop,Café,Fast Food Restaurant,Tennis Court,Basketball Court,Grocery Store,Auto Garage
11302,CR0 5AR,51.357202,-0.033270,Croydon,Selsdon & Addington Village,Addington Village,0.114989,3,CR0,4,English Restaurant,Gas Station,Yoshoku Restaurant,Fast Food Restaurant,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit
30459,E1 7PS,51.515023,-0.073249,Tower Hamlets,Spitalfields & Banglatown,Aldgate East,0.018315,1,E1,4,Coffee Shop,Hotel,Pub,Pizza Place,Indian Restaurant,Cocktail Bar,Gym / Fitness Center,Japanese Restaurant,Café,Italian Restaurant


In [197]:
cluster0 = london_merged[london_merged['ClusterLabels'] == 0]
cluster0.head()

,Postcode,Latitude,Longitude,District,Ward,NearestStation,DistanceToStation,London zone,Postcode district,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16105,CR4 3QE,51.401492,-0.178504,Merton,Cricket Green,Belgrave Walk,0.058935,3,CR4,0,Pub,Garden,Café,Yoshoku Restaurant,Farm,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
148049,NW3 2AL,51.550126,-0.163862,Camden,Hampstead Town,Belsize Park,0.010602,2,NW3,0,Café,Pub,Hotel,Gym / Fitness Center,Coffee Shop,Bakery,Pizza Place,Pharmacy,Tapas Restaurant,Bookstore
47070,E3 3JJ,51.524840,-0.011682,Tower Hamlets,Bromley North,Bromley-by-Bow,0.000000,2,E3,0,Historic Site,Café,Grocery Store,Supermarket,Mobile Phone Shop,Waterfront,Pub,Canal Lock,Event Service,Ethiopian Restaurant
17871,CR5 3ED,51.305491,-0.153571,Croydon,Coulsdon Town,Chipstead,1.180200,6,CR5,0,Café,Yoshoku Restaurant,Farmers Market,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit
304405,W4 3HF,51.480781,-0.267013,Hounslow,Chiswick Riverside,Chiswick,0.000000,3,W4,0,Pub,Cricket Ground,Café,Gym,Restaurant,Gym / Fitness Center,Flea Market,Fish & Chips Shop,Exhibit,Duty-free Shop


In [198]:
london_merged['ClusterLabels'].unique()

array([4, 8, 2, 1, 6, 3, 9, 0, 7, 5])